# MDATP - Use query from Github Repository


## Replace variables with pertaining to your environment

In [40]:
############ This is the credentials used to authenticate with the Graph API ###################
$credentials = @{
    tenantDomain = "domain.com"
    tenantId = "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxx"        ## AAD Tenant ID
    client_id = "ae4fa870-23af-4ae5-a6ed-xxxxx"     ## Application Client ID
    client_secret = "x~xxxxxxx~xxxxx~xxxxxxxxx-xxx"   ## Application Client Secret
}

## Import required modules and credentials 

In [8]:
$modules = @('pp-core','graph') ## These are the modules to import 
$credentialPath = "~/.credentials/graph.credential" ## Where to store credentials

################## Import Graph Authentication Module #############################
ForEach ($module in $modules) {
    Import-Module "~/Notebooks/Powershell-Playground/PowerShell/custom-modules/$module" -Force  ## import module 
    Get-Command -Module "$module"                                                        ## list commands in this module 
}

$credentials = Get-StoredCredentials -credentialPath $credentialPath


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

 ·········


## Authenticate with Graph API and get access Token for MDATP Resource

In [9]:
## Get a token if authenticates properly.
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.securitycenter.windows.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
}; $authParams

$tokenResponse = Invoke-GraphAuthentication -authParams $authParams
$tokenResponse


Name                           Value
----                           -----
client_id                      ae4fa870-23af-4ae5-a6ed-5ab1811858c7
tenantId                       de40cf7e-ad5f-4245-a317-14be39cbb0ef
resource                       https://api.securitycenter.windows.com
client_secret                  5~Q5En6ecx9d~ZtuqPf~YRBLw5S858-7WS
grant_type                     client_credentials

Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/de40cf7e-ad5f-4245-a317-14be39cbb0ef/oauth2/token


Received Token!
Connected and Access Token received and will expire 

token_type     : Bearer
expires_in     : 3599
e

## Pull raw query from Github
Be sure to provide the <font color=Green><b>RAW Content</b></font> and not a the github link of the source

In [3]:
$rawGithubLink = "https://raw.githubusercontent.com/microsoft/Microsoft-threat-protection-Hunting-Queries/master/Discovery/URL%20Detection.txt"
$Response = Invoke-WebRequest -Method GET -URI $rawGithubLink
$Response.Content

// This query finds network communication to specific URL
// Please note that in line #7 it filters RemoteUrl using has operator, which looks for a "whole term" and runs faster.
// Example: RemoteUrl has "microsoft" matches "www.microsoft.com" but not "microsoftonline.com"
let partialRemoteUrlToDetect = "microsoft.com"; // Change this to a URL you'd like to find machines connecting to
DeviceNetworkEvents  
| where Timestamp > ago(7d)
and RemoteUrl has partialRemoteUrlToDetect // Can be changed to "contains" operator as explained above
| project Timestamp, DeviceName, DeviceId, ReportId
| top 100 by Timestamp desc



## Put your Hunting Query here

In [17]:
## Put your query on the 3rd Line below $Query and make sure it ends with #@
$Query = @"
DeviceInfo
"@

$body = @{Query = $Query} | ConvertTo-JSON

Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body


Query
DeviceInfo
JSON
{
  "Query": "DeviceInfo"
}


## Query the MDATP API Grab report schema 

In [18]:
$URI = "https://api.securitycenter.windows.com/api/advancedqueries/run"
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" }
$authHeader
$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Result


Name                           Value
----                           -----
Accept                         application/json
Content-Type                   application/json
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlNzWnNCTmhaY0Yz…


Stats   : @{ExecutionTime=0.04691; resource_usage=; dataset_statistics=System.Object[]}
Schema  : {@{Name=Timestamp; Type=DateTime}, @{Name=DeviceId; Type=String}, @{Name=DeviceName; 
          Type=String}, @{Name=ClientVersion; Type=String}…}
Results : {@{Timestamp=6/12/2020 6:19:15 AM; DeviceId=bffd249f248a1d1b95e5f90922c9a1df776de35f; 
          DeviceName=jing-lab; ClientVersion=10.3720.16299.2010; PublicIP=173.224.104.67; 
          OSArchitecture=; OSPlatform=; OSBuild=; IsAzureADJoined=; DeviceObjectId=; 
          LoggedOnUsers=[{"UserName":"Administrator","DomainName":"JING-LAB"}]; 
          RegistryDeviceTag=; OSVersion=; MachineGroup=UnassignedGroup; ReportId=7890; 
          AdditionalFields=[]}, @{Times

## Parse report

In [7]:
$includeColumns = @('Timestamp','DeviceName','ActionType','RemoteIP')
$Result.Results | Select-Object -Property $includeColumns


Timestamp             DeviceName        ActionType RemoteIP
---------             ----------        ---------- --------
6/24/2020 7:12:32 PM  win10-oobe-test              
6/24/2020 7:12:30 PM  win10-oobe-test              
6/24/2020 7:12:29 PM  win10-oobe-test              
6/24/2020 7:02:17 PM  jingtoso-desktop1            
6/24/2020 6:48:40 PM  win10-oobe-test              
6/24/2020 6:40:39 PM  win10-oobe-test              
6/24/2020 6:38:11 PM  jingtoso-desktop1            
6/24/2020 6:18:44 PM  win10-oobe-test              
6/24/2020 6:18:24 PM  win10-oobe-test              
6/24/2020 6:18:23 PM  win10-oobe-test              
6/24/2020 6:18:16 PM  win10-oobe-test              
6/24/2020 6:18:15 PM  win10-oobe-test              
6/24/2020 6:13:13 PM  win10-oobe-test              
6/24/2020 5:58:32 PM  win10-oobe-test              
6/24/2020 5:57:34 PM  win10-oobe-test              
6/24/2020 5:36:50 PM  jing-lab                     
6/24/2020 5:24:09 PM  win10-oobe-test          